In [14]:
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import numpy as np
import cv2

# read image+crop image
path=r'C:/Users/Keitaro Ninomiya/Box/Research Notes (keitaro2@illinois.edu)/Tokyo_Jobs/Raw_Data/Metropolitan_DA/1938/Line/'
img = cv2.imread(path+"Page050.jpg")

hh, ww = img.shape[:2]
# convert to grayscale
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
# threshold gray image
thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)[1]

# count number of non-zero pixels in each row. 
countCol = np.count_nonzero(thresh, axis=0)
countRow = np.count_nonzero(thresh, axis=1)

### Find location of biggest jumps/falls.
###Top and Bottom
peaks, _ = find_peaks(countRow, distance=15)
max1Height=np.sort(abs(np.diff(countRow[peaks])))[-1]
max2Height=np.sort(abs(np.diff(countRow[peaks])))[-2]
X=peaks[ : -1][abs(np.diff(countRow[peaks]))==max1Height][0]
Y=peaks[ : -1][abs(np.diff(countRow[peaks]))==max2Height][0]
Top=min([X,Y])
Btm=max([X,Y])

###Left and Right
peaks, _ = find_peaks(countCol, distance=15)
HeightLeft=abs(np.diff(countCol[peaks][peaks<ww//2]))
HeightRight=abs(np.diff(countCol[peaks][peaks>ww//2]))
max1Height=np.sort(HeightLeft)[-1]
max2Height=np.sort(HeightRight)[-1]
V=peaks[ : -1][abs(np.diff(countCol[peaks]))==max1Height][0]
Z=peaks[ : -1][abs(np.diff(countCol[peaks]))==max2Height][0]
Left=min([V,Z])
Right=max([V,Z])

print((Top,Btm, Left,Right))

cropped=img[Top:Btm, Left:Right] 
cv2.imshow("Result", cropped)
cv2.waitKey(0)


(74, 474, 77, 1202)


-1

In [2]:
#Code for cutting image Side-by-Side
# read image+crop image
crop_img=img[Top:Btm, Left:Right]
HH, WW = crop_img.shape[:2]
# convert to grayscale
gray = cv2.cvtColor(crop_img,cv2.COLOR_BGR2GRAY)
# threshold gray image
thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)[1]

# count number of non-zero pixels in each column and row. 
countCol = np.count_nonzero(thresh, axis=0)
countRow = np.count_nonzero(thresh, axis=1)

### This finds the height of the lowest valley
peaks, _ = find_peaks(-countCol, distance=15)
maxHeight=np.sort(abs(np.diff(countCol[peaks][(peaks<4*WW//7) & (peaks>3*WW//7)])))[-1]
Border=peaks[ : -1][abs(np.diff(countCol[peaks]))==maxHeight][0]

### loop over contours and get bounding boxes and ycenter and draw horizontal line at ycenter
result = crop_img.copy()
LeftImage=result[0:HH,0:Border]
RighImage=result[0:HH,Border:WW]
cv2.imshow("LeftImage", LeftImage)
cv2.imshow("RightImage", RighImage)
cv2.waitKey(0)

cv2.imwrite("C:/Users/Keitaro Ninomiya/Box/Research Notes (keitaro2@illinois.edu)/Tokyo_Jobs/Data_Collection/Temp_Folder/LeftImage.png", LeftImage)

False

In [2]:
import requests
import uuid
import time
import json

api_url = 'http://clovaocr-api-jp.ncloud.com/external/v1/1881/ea637829be08aa3546739ce13af6fdacab23a220835cc7bdd7b77379f7c3299f'
secret_key = 'aWxtREpqUExnSnZZRk1IVmVDdk56ZElkbW9LeVVEUkc='
image_file = "C:/Users/Keitaro Ninomiya/Box/Research Notes (keitaro2@illinois.edu)/Tokyo_Jobs/Data_Collection/Temp_Folder/LeftImage.png"

request_json = {
'images': [
{
'format': 'png/jpg/pdf',
'name': 'demo'
}
],
'requestId': str(uuid.uuid4()),
'version': 'V2',
'timestamp': int(round(time.time() * 1000)),
'lang':'ja'
}

payload = {'message': json.dumps(request_json).encode('UTF-8')}
files = [
('file', open(image_file,'rb'))
]
headers = {
'X-OCR-SECRET': secret_key
}

response = requests.request("POST", api_url, headers=headers, data = payload, files = files)

print(response.text.encode('utf8'))


ConnectionError: HTTPConnectionPool(host='clovaocr-api-jp.ncloud.com', port=80): Max retries exceeded with url: /external/v1/1881/ea637829be08aa3546739ce13af6fdacab23a220835cc7bdd7b77379f7c3299f (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001BC7DBFCFA0>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

In [3]:
#Code for Cutting image Up-to-Down for multi-row image
#Right page
import cv2
import numpy as np
from scipy.signal import find_peaks

# read image+crop image
img = crop_img.copy()[0:HH,Border:WW]
#Update Height & Width
hh, ww = img.shape[:2]
# convert to grayscale
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
# threshold gray image
thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)[1]
# count number of non-zero pixels in each column and row. 
countRow = np.count_nonzero(thresh, axis=1)

### Looks for valleys: Choose the valley closest to the middle line.
peaks, _ = find_peaks(-countRow, distance=30)

### threshold count at Thres
count_thresh = countRow.copy()
count_thresh[peaks] = 255
count_thresh[count_thresh!=255] = 0
count_thresh = count_thresh.astype(np.uint8)

### get contours
contours = cv2.findContours(count_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0] if len(contours) == 2 else contours[1]

### loop over contours and get bounding boxes and ycenter and draw horizontal line at ycenter
result = crop_img.copy()
for cntr in contours: #Possibly replace this by nearest contour
    x,y,w,h = cv2.boundingRect(cntr)
    ycenter = y
    if abs(ycenter-hh//2)<20:
        CenterLine = y+20
        cv2.line(result, (0,CenterLine), (ww,CenterLine), (255, 0, 0), 1)
    else:
        continue

TopImage=result[0:CenterLine,Border:WW]
BtmImage=result[CenterLine:hh,Border:WW]

# display results
cv2.imshow("TopImage", TopImage)
cv2.imshow("BtmImage", BtmImage)
cv2.waitKey(0)


-1

In [4]:
#Code for adjusting angle: Right page
import imutils
import matplotlib.pyplot as plt

def Rotate_Right(RighImage):    
    #### Bench Marking ####
    gray = cv2.cvtColor(RighImage,cv2.COLOR_BGR2GRAY)
    # threshold gray image
    thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)[1]
    ## count number of non-zero pixels in each column and row. 
    countCol = np.count_nonzero(thresh, axis=0)
    ## This finds the locations of valleys
    peaks, _ = find_peaks(countCol, distance=30)
    
    res = res = np.mean(np.diff(peaks))
    print("Initial average distance is"+str(res))

    #Rotate image
    for i in range(1,10):
        Rotimage = imutils.rotate(RighImage, angle=0.5*i)
        
        gray = cv2.cvtColor(Rotimage,cv2.COLOR_BGR2GRAY)
        thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)[1]
        countCol = np.count_nonzero(thresh, axis=0)
        peaks, _ = find_peaks(countCol, distance=30)
        new_res = np.mean(np.diff(peaks))
        
        if res>new_res:
            print("Updating tilts, New distance is "+str(new_res))
            res=new_res
            continue
        else:
            print("Finished: Picture was rotated by "+str(0.5*i)+" degrees. Final distance is"+str(res)+". Next was"+str(new_res))
            return [Rotimage,i]
            break

img=Rotate_Right(TopImage)[0]
degrees=Rotate_Right(TopImage)[1]*0.5
cv2.imshow("image",img)
cv2.waitKey(0)

Initial average distance is41.142857142857146
Finished: Picture was rotated by 0.5 degrees. Final distance is41.142857142857146. Next was41.84615384615385
Initial average distance is41.142857142857146
Finished: Picture was rotated by 0.5 degrees. Final distance is41.142857142857146. Next was41.84615384615385


-1

In [22]:
#Code for Adding Grid
##Right page
img = crop_img.copy()[0:CenterLine,Border:WW]
img = imutils.rotate(img, angle=degrees)

#Identify grid location
## convert to grayscale
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
# threshold gray image
thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)[1]

## count number of non-zero pixels in each column and row. 
countCol = np.count_nonzero(thresh, axis=0)
countRow = np.count_nonzero(thresh, axis=1)

## Column lines
### This finds the height of the smallest peak
peaks, _ = find_peaks(countCol, distance=15)
Thres=min(countCol[peaks])
### threshold count at Thres
count_thresh = countCol.copy()
count_thresh[peaks] = 255
count_thresh[count_thresh!=255] = 0
count_thresh = count_thresh.astype(np.uint8)

### get contours
contours = cv2.findContours(count_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0] if len(contours) == 2 else contours[1]

### loop over contours and get bounding boxes and ycenter and draw horizontal line at ycenter
result = img.copy()
for cntr in contours:
    x,y,w,h = cv2.boundingRect(cntr)
    ycenter = y+h//2
    cv2.line(result, (ycenter,0), (ycenter,hh), (255, 0, 0), 1)

### Row lines ###
peaks, _ = find_peaks(-countRow, distance=50)
Thres=min(countRow[peaks])
### threshold count at Thres
count_thresh = countRow.copy()
count_thresh[countRow==Thres] = 255
count_thresh[countRow<Thres] = 0
count_thresh = count_thresh.astype(np.uint8)

### get contours
contours = cv2.findContours(count_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0] if len(contours) == 2 else contours[1]

### loop over contours and get bounding boxes and ycenter and draw horizontal line at ycenter
for cntr in contours:
    x,y,w,h = cv2.boundingRect(cntr)
    ycenter = y+h//2
    cv2.line(result, (0,ycenter), (ww,ycenter), (255, 0, 0), 1)

# display results
cv2.imshow("RESULT", result)
cv2.waitKey(0)

cv2.imwrite(r'C:/Users/Keitaro Ninomiya/Desktop/TopImage_example_quality.png', result,[cv2.IMWRITE_JPEG_QUALITY, 100])

True